# Pandas 데이터 합성

pandas는 두 개 이상의 데이터프레임을 하나로 합치는 데이터 합성을 지원한다.

## Merge

Merge 명령은 두 데이터프레임의 공통 열 혹은 인덱스를 기준으로 데이터베이스 테이블 조인(join)과 같이 두 개의 테이블을 합친다.

### merge 예제 1

In [1]:
import numpy as np
import pandas as pd

df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'], 'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [2]:
df2 = pd.DataFrame({'key':['a','b','d'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


merge 명령으로 두 데이터프레임 df1, df2 를 합치면 공통 열인 key 열을 기준으로 데이터를 찾아서 합친다.

In [3]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


outer 방식은 null 이 있어도 양쪽 데이터를 모두 보여준다.

In [4]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


left, right 방식은 첫번째, 혹은 두번째 데이터프레임을 모두 보여준다.

In [5]:
pd.merge(df1, df2, how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [6]:
pd.merge(df1, df2, how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


### merge 예제 2

In [7]:
df1 = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                    'key2': ['one', 'two', 'one'], 
                    'lval': [1, 2, 3]})
df1

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [8]:
df2 = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                    'key2': ['one', 'one', 'one', 'two'],
                    'rval': [4, 5, 6, 7]})
df2

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


이름이 같은 열은 모두 기준 열로 사용된다.

In [9]:
pd.merge(df1, df2, how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


기준 열은 on 인수로 명시적 설정이 가능하다.

In [10]:
pd.merge(df1, df2, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


이 때 기준 열이 아니면서 이름이 같은 열에는 _x 또는 _y 와 같은 접미사가 붙는다.

In [11]:
pd.merge(df1, df2, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


suffixes 인수로 접미사를 직접 설정할 수도 있다.

In [12]:
pd.merge(df1, df2, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### merge 예제 3

기준 열을 각각의 데이터프레임에 대해 다르게 정하려면 left_on, right_on 인수를 사용한다.

In [13]:
df1 = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                    'key2': ['one', 'two', 'one'], 
                    'lval': [1, 2, 3]})
df1

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [14]:
df2 = pd.DataFrame({'k1': ['foo', 'foo', 'bar', 'bar'],
                    'k2': ['one', 'one', 'one', 'two'],
                    'rval': [4, 5, 6, 7]})
df2

,k1,k2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [15]:
pd.merge(df1, df2, left_on='key1', right_on="k1")

,key1,key2,lval,k1,k2,rval
0,foo,one,1,foo,one,4
1,foo,one,1,foo,one,5
2,foo,two,2,foo,one,4
3,foo,two,2,foo,one,5
4,bar,one,3,bar,one,6
5,bar,one,3,bar,two,7


### merge 예제 4

일반 데이터 열이 아닌 인덱스를 기준열로 사용하려면 left_index 또는 right_index 인수를 True 로 설정한다.

In [16]:
df1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
df1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [17]:
df2 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
df2

,group_val
a,3.5
b,7.0


In [18]:
pd.merge(df1, df2, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


### merge 예제 5

In [19]:
df1 = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                    'key2': [2000, 2001, 2002, 2001, 2002],
                    'data': np.arange(5.)})
df1

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [20]:
df2 = pd.DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
df2

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [21]:
pd.merge(df1, df2, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


### merge 예제 6

In [22]:
df1 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], 
                   index=['a', 'c', 'e'], 
                   columns=['Ohio', 'Nevada'])
df1

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [23]:
df2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], 
                   index=['b', 'c', 'd', 'e'], 
                   columns=['Missouri', 'Alabama'])
df2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [24]:
pd.merge(df1, df2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### join 메서드

merge 명령어 대신 join 메서드를 사용할 수도 있다.

In [25]:
df1.join(df2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


## Concat

concat 명령을 사용하면 기준 열(key column)을 사용하지 않고 단순히 데이터를 추가한다.

기본적으로는 아래에 데이터 행을 덧붙이지만 (인덱스가 중복됨) axis=1로 인수를 설정하면 인덱스 기준으로 옆으로 데이터 열를 덧붙인다.

In [26]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [27]:
s1

a    0
b    1
dtype: int64

In [28]:
s2 

c    2
d    3
e    4
dtype: int64

In [29]:
s3

f    5
g    6
dtype: int64

In [30]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [31]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [32]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [33]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [34]:
pd.concat([df1, df2])

,four,one,three,two
a,NaN,0.0,NaN,1.0
b,NaN,2.0,NaN,3.0
c,NaN,4.0,NaN,5.0
a,6.0,NaN,5.0,NaN
c,8.0,NaN,7.0,NaN


In [35]:
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0
